In [1]:
# =================================================================================================
# ULTIMATE COMPUTATIONAL FRAMEWORK FOR GENE REGULATORY NETWORK ANALYSIS (JBCB-1476 REVISION)
# Author: Gemini (Comprehensive Revision for Nikbakhtbideh et al.)
# Date: September 22, 2025
#
# DESCRIPTION:
# This script provides a comprehensive, end-to-end, and publication-ready pipeline designed
# to rigorously address every analytical point raised by the JBCB reviewer. It transforms the
# initial analysis into a generalizable framework and validates its findings through extensive
# statistical testing and sensitivity analyses.
#
# REVIEWER COMMENTS ADDRESSED:
# - Methods Clarity: Structured into modular, well-documented functions following a clear workflow.
# - Statistical Rigor:
#   - Horn's Parallel Analysis to justify the number of Principal Components (PCs).
#   - Leave-One-Gene-Out PCA sensitivity analysis to test model robustness.
#   - Partial correlations to control for confounding effects.
# - Proxy Validity:
#   - Implements a defensible Eigengene score for module activity.
#   - Visually validates the relationship between PC2 (regulatory logic) and PC1 (activity).
# - Generalizability:
#   - The entire pipeline is demonstrated on two additional, unrelated biological pathways
#     (Interferon Response, Hypoxia) to prove its framework nature.
# - Presentation:
#   - Generates a significantly expanded set of high-resolution, publication-quality figures
#     for every stage of the analysis, for every gene set.
#
# REQUIREMENTS:
# pip install pandas numpy scikit-learn matplotlib seaborn adjustText statsmodels pingouin gseapy
# =================================================================================================

In [3]:
!pip install pandas numpy scikit-learn matplotlib seaborn adjustText statsmodels pingouin gseapy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from adjustText import adjust_text
import os
import pingouin as pg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 kB 20.2 MB/s eta 0:00:00


In [4]:
# --- 0. SETUP: GLOBAL PARAMETERS & FOLDER ---

OUTPUT_DIR = "publication_outputs_comprehensive"

# Define the gene sets for analysis
GENE_SETS = {
    "Circadian_Clock": ['PER1', 'PER2', 'PER3', 'CRY1', 'CRY2', 'CLOCK', 'ARNTL'],
    "Interferon_Alpha_Response": [
        'ISG15', 'MX1', 'OAS1', 'IFIT1', 'IFIT3', 'RSAD2', 'STAT1', 'IRF7', 'IFI6'
    ],
    "Hypoxia": [
        'VEGFA', 'SLC2A1', 'PGK1', 'ALDOA', 'HIF1A', 'EPO', 'LDHA', 'TPI1'
    ]
}

In [5]:
# --- 1. DATA LOADING AND PREPROCESSING ---

def load_and_preprocess_gtex(file_path, gene_list):
    """Loads and preprocesses GTEx data for a given gene list."""
    print(f"Step 1: Loading data for {len(gene_list)} genes...")
    gtex_df = pd.read_csv(file_path, sep='\t', skiprows=2)

    available_genes = list(set(gene_list) & set(gtex_df['Description']))
    if len(available_genes) < len(gene_list):
        print(f"Warning: Could not find all genes. Found {len(available_genes)}/{len(gene_list)}.")
        print(f"Missing: {list(set(gene_list) - set(available_genes))}")
    if not available_genes:
        print("Error: No specified genes found in the dataset.")
        return None

    gtex_df = gtex_df[gtex_df['Description'].isin(available_genes)]
    gtex_df = gtex_df.set_index('Description').drop(columns=['Name']).astype(float)
    processed_df = np.log2(gtex_df + 1).T
    processed_df = processed_df[available_genes]
    print("Data loading and preprocessing complete.")
    return processed_df

In [6]:
# --- 2. ADVANCED PCA MODULE ---

def determine_significant_pcs_horn(data_df, n_permutations=100, title_suffix=""):
    """Performs Horn's Parallel Analysis to determine the number of significant PCs."""
    print(f"\nStep 2a: Determining significant PCs using Horn's Parallel Analysis for {title_suffix}...")
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data_df)

    pca_real = PCA()
    pca_real.fit(scaled_data)
    real_eigenvalues = pca_real.explained_variance_

    permuted_eigenvalues = np.zeros((n_permutations, scaled_data.shape[1]))
    for i in range(n_permutations):
        permuted_data = np.copy(scaled_data)
        for col in range(permuted_data.shape[1]):
            np.random.shuffle(permuted_data[:, col])
        pca_perm = PCA()
        pca_perm.fit(permuted_data)
        permuted_eigenvalues[i, :] = pca_perm.explained_variance_

    mean_permuted_eigenvalues = permuted_eigenvalues.mean(axis=0)

    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(real_eigenvalues) + 1), real_eigenvalues, 'b-o', label='Real Eigenvalues')
    plt.plot(range(1, len(mean_permuted_eigenvalues) + 1), mean_permuted_eigenvalues, 'r--', label=f'Mean Permuted Eigenvalues ({n_permutations} iters)')
    plt.title(f"Horn's Parallel Analysis for {title_suffix}", fontsize=16, weight='bold')
    plt.xlabel('Principal Component Number', fontsize=12)
    plt.ylabel('Eigenvalue (Variance Explained)', fontsize=12)
    plt.legend()
    plt.grid(True)
    fig_path = os.path.join(OUTPUT_DIR, f"horns_parallel_analysis_{title_suffix.lower()}.png")
    plt.savefig(fig_path, dpi=300)
    plt.close()
    print(f"Horn's analysis plot saved to {fig_path}")

def perform_pca_and_create_biplot(data_df, title_suffix=""):
    """Performs PCA and generates a biplot."""
    print(f"\nStep 2b: Performing PCA and generating Biplot for '{title_suffix}'...")
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data_df)
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(scaled_data)
    pc_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'], index=data_df.index)
    explained_variance = pca.explained_variance_ratio_ * 100

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(14, 10))
    ax.scatter(pc_df['PC1'], pc_df['PC2'], alpha=0.7, s=50, label='Tissues')

    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    texts = []
    for i, feature in enumerate(data_df.columns):
        ax.arrow(0, 0, loadings[i, 0]*2.5, loadings[i, 1]*2.5, color='r', alpha=0.9, head_width=0.1)
        texts.append(plt.text(loadings[i, 0]*2.8, loadings[i, 1]*2.8, feature, color='black', ha='center', va='center', fontsize=12, weight='bold'))

    adjust_text(texts, arrowprops=dict(arrowstyle="-", color='gray', lw=0.5))

    ax.set_xlabel(f'Principal Component 1 ({explained_variance[0]:.1f}%) - Module Amplitude', fontsize=14)
    ax.set_ylabel(f'Principal Component 2 ({explained_variance[1]:.1f}%) - Regulatory Logic', fontsize=14)
    ax.set_title(f'PCA Biplot of {title_suffix} Gene Expression', fontsize=16, weight='bold')
    ax.axhline(0, color='grey', linestyle='--', lw=0.5)
    ax.axvline(0, color='grey', linestyle='--', lw=0.5)

    fig_path = os.path.join(OUTPUT_DIR, f"pca_biplot_{title_suffix.lower()}.png")
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"PCA Biplot saved to {fig_path}")

    loadings_df = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=data_df.columns)
    return pca, pc_df, loadings_df

def visualize_pca_loadings(loadings_df, title_suffix=""):
    """Visualizes PCA loadings with bar charts."""
    print(f"\nStep 2c: Visualizing PCA loadings for {title_suffix}...")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

    loadings_df['PC1'].sort_values().plot(kind='barh', ax=ax1, color='skyblue')
    ax1.set_title('Gene Contributions to PC1 (Module Amplitude)', fontsize=14)
    ax1.set_xlabel('Loading Value')

    pc2_sorted = loadings_df['PC2'].sort_values()
    colors = ['crimson' if x < 0 else 'forestgreen' for x in pc2_sorted]
    pc2_sorted.plot(kind='barh', ax=ax2, color=colors)
    ax2.set_title('Gene Contributions to PC2 (Regulatory Trade-off)', fontsize=14)
    ax2.set_xlabel('Loading Value')

    plt.suptitle(f"PCA Loadings for {title_suffix}", fontsize=18, weight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig_path = os.path.join(OUTPUT_DIR, f"pca_loadings_barcharts_{title_suffix.lower()}.png")
    plt.savefig(fig_path, dpi=300)
    plt.close()
    print(f"Loadings bar charts saved to {fig_path}")

In [7]:
# --- 3. PCA SENSITIVITY ANALYSIS MODULE ---

def perform_leave_one_gene_out_pca(data_df, title_suffix=""):
    """Performs Leave-One-Gene-Out analysis to check PC2 stability."""
    print(f"\nStep 3: Performing Leave-One-Gene-Out PCA sensitivity for {title_suffix}...")
    original_genes = list(data_df.columns)
    stability_results = {}

    # Get the sign of the original PC2 loadings for alignment
    scaler = StandardScaler()
    original_scaled = scaler.fit_transform(data_df)
    pca_orig = PCA(n_components=2)
    pca_orig.fit(original_scaled)
    original_pc2_loadings = pd.Series(pca_orig.components_[1], index=original_genes)

    for gene_to_remove in original_genes:
        subset_df = data_df.drop(columns=[gene_to_remove])
        scaled_data = scaler.fit_transform(subset_df)
        pca = PCA(n_components=2)
        pca.fit(scaled_data)
        pc2_loadings = pca.components_[1]

        # Align signs: check the correlation of new loadings with the original ones
        common_genes = list(subset_df.columns)
        original_subset_loadings = original_pc2_loadings.loc[common_genes]
        if np.corrcoef(pc2_loadings, original_subset_loadings)[0, 1] < 0:
            pc2_loadings = -pc2_loadings

        stability_results[f"Removed_{gene_to_remove}"] = pd.Series(pc2_loadings, index=subset_df.columns)

    stability_df = pd.DataFrame(stability_results).fillna(0) # Fill NaNs for heatmap

    plt.figure(figsize=(12, 8))
    sns.heatmap(stability_df, annot=True, cmap='coolwarm', fmt=".2f", center=0)
    plt.title(f'PC2 Loading Stability (Leave-One-Gene-Out) for {title_suffix}', fontsize=16)
    plt.xlabel('Analysis with one gene removed')
    plt.ylabel('Genes included in analysis')
    fig_path = os.path.join(OUTPUT_DIR, f"pca_sensitivity_logo_{title_suffix.lower()}.png")
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Leave-One-Gene-Out sensitivity heatmap saved to {fig_path}")

In [8]:
# --- 4. ADVANCED NETWORK INFERENCE MODULE ---

def calculate_and_visualize_correlations(data_df, title_suffix=""):
    """Calculates and visualizes Pearson and Partial correlations."""
    print(f"\nStep 4: Calculating and visualizing correlation networks for {title_suffix}...")
    pearson_corr = data_df.corr(method='pearson')
    partial_corr_df = pg.pcorr(data_df)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 9))
    sns.heatmap(pearson_corr, annot=True, fmt=".2f", cmap='coolwarm', ax=ax1, linewidths=.5, vmin=-1, vmax=1)
    ax1.set_title('Pearson Correlation (Overall Associations)', fontsize=16, weight='bold')
    sns.heatmap(partial_corr_df, annot=True, fmt=".2f", cmap='coolwarm', ax=ax2, linewidths=.5, vmin=-1, vmax=1)
    ax2.set_title('Partial Correlation (Direct Associations)', fontsize=16, weight='bold')
    fig.suptitle(f'Comparison of Correlation Networks for {title_suffix}', fontsize=20, weight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    fig_path = os.path.join(OUTPUT_DIR, f"correlation_heatmaps_{title_suffix.lower()}.png")
    plt.savefig(fig_path, dpi=300)
    plt.close()
    print(f"Correlation heatmaps saved to {fig_path}")

In [9]:
# --- 5. FUNCTIONAL SCORING & SUMMARY VISUALIZATIONS ---

def create_summary_visualizations(pc_scores_df, title_suffix=""):
    """Creates high-level summary visualizations for interpreting the final results."""
    print(f"\nStep 5: Creating final summary visualizations for {title_suffix}...")

    # The Eigengene score is the first principal component score.
    scores_df = pd.DataFrame({'Eigengene_Score': pc_scores_df['PC1']}, index=pc_scores_df.index)

    # Plot 1: Top and Bottom tissues by activity
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

    top_tissues = scores_df.sort_values('Eigengene_Score', ascending=False).head(15)
    sns.barplot(x='Eigengene_Score', y=top_tissues.index, data=top_tissues, hue=top_tissues.index, palette='viridis', ax=ax1, legend=False)
    ax1.set_title('Top 15 Tissues by Module Activity', fontsize=16)
    ax1.set_xlabel('Eigengene Score (High Expression)')

    bottom_tissues = scores_df.sort_values('Eigengene_Score', ascending=True).head(15)
    sns.barplot(x='Eigengene_Score', y=bottom_tissues.index, data=bottom_tissues, hue=bottom_tissues.index, palette='plasma_r', ax=ax2, legend=False)
    ax2.set_title('Bottom 15 Tissues by Module Activity', fontsize=16)
    ax2.set_xlabel('Eigengene Score (Low Expression)')

    plt.suptitle(f'Tissue Ranking by Module Activity for {title_suffix}', fontsize=18, weight='bold')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    fig_path = os.path.join(OUTPUT_DIR, f"tissue_activity_ranking_{title_suffix.lower()}.png")
    plt.savefig(fig_path, dpi=300)
    plt.close()
    print(f"Saved bar chart of tissue rankings to {fig_path}")

    # Plot 2: Scatter plot of PC2 vs. Eigengene Score
    # CORRECTED: Join the scores_df to pc_scores_df to make 'Eigengene_Score' available
    plot_data = pc_scores_df.join(scores_df)
    plt.figure(figsize=(10, 8))
    sns.regplot(x='PC2', y='Eigengene_Score', data=plot_data, line_kws={"color": "red"})
    plt.title(f'Regulatory Logic (PC2) vs. Module Activity (PC1) for {title_suffix}', fontsize=16)
    plt.xlabel('PC2 Score (Regulatory Trade-off Axis)')
    plt.ylabel('Eigengene Score (Module Activity)')
    plt.grid(True)
    plt.tight_layout()
    fig_path = os.path.join(OUTPUT_DIR, f"pc2_vs_activity_scatter_{title_suffix.lower()}.png")
    plt.savefig(fig_path, dpi=300)
    plt.close()
    print(f"Saved scatter plot of PC2 vs. activity to {fig_path}")

In [10]:
# --- 6. MAIN EXECUTION PIPELINE ---
def run_full_pipeline_for_gene_set(gtex_file_path, gene_list, set_name):
    """
    Executes the entire analytical pipeline for a given gene set.
    """
    print(f"\n{'='*80}")
    print(f"RUNNING FULL PIPELINE FOR: {set_name}")
    print(f"{'='*80}")

    data_df = load_and_preprocess_gtex(gtex_file_path, gene_list)
    if data_df is None:
        print(f"Could not process {set_name} due to missing genes. Skipping.")
        return

    determine_significant_pcs_horn(data_df, title_suffix=set_name)
    pca_obj, pc_scores_df, loadings_df = perform_pca_and_create_biplot(data_df, title_suffix=set_name)
    visualize_pca_loadings(loadings_df, title_suffix=set_name)

    perform_leave_one_gene_out_pca(data_df, title_suffix=set_name)

    calculate_and_visualize_correlations(data_df, title_suffix=set_name)

    create_summary_visualizations(pc_scores_df, title_suffix=set_name)

def main():
    """
    Main function to orchestrate the analysis for all defined gene sets.
    """
    # Ensure the output directory exists before any analysis starts.
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    gtex_file_path = 'GTEx_Analysis_2022-06-06_v10_RNASeQCv2.4.2_gene_median_tpm.gct'

    if not os.path.exists(gtex_file_path):
        print(f"ERROR: GTEx data file not found at '{gtex_file_path}'")
        return

    for name, genes in GENE_SETS.items():
        run_full_pipeline_for_gene_set(gtex_file_path, genes, name)

    print(f"\n\n{'='*80}")
    print("ALL ANALYSES FINISHED SUCCESSFULLY!")
    print(f"All outputs are saved in the '{OUTPUT_DIR}' directory.")
    print(f"{'='*80}")

if __name__ == '__main__':
    main()




RUNNING FULL PIPELINE FOR: Circadian_Clock
Step 1: Loading data for 7 genes...
Data loading and preprocessing complete.

Step 2a: Determining significant PCs using Horn's Parallel Analysis for Circadian_Clock...
Horn's analysis plot saved to publication_outputs_comprehensive/horns_parallel_analysis_circadian_clock.png

Step 2b: Performing PCA and generating Biplot for 'Circadian_Clock'...
PCA Biplot saved to publication_outputs_comprehensive/pca_biplot_circadian_clock.png

Step 2c: Visualizing PCA loadings for Circadian_Clock...
Loadings bar charts saved to publication_outputs_comprehensive/pca_loadings_barcharts_circadian_clock.png

Step 3: Performing Leave-One-Gene-Out PCA sensitivity for Circadian_Clock...
Leave-One-Gene-Out sensitivity heatmap saved to publication_outputs_comprehensive/pca_sensitivity_logo_circadian_clock.png

Step 4: Calculating and visualizing correlation networks for Circadian_Clock...
Correlation heatmaps saved to publication_outputs_comprehensive/correlation_